In [204]:
# Import stuff
using CSV
using DataFrames
using BenchmarkTools
# import Base.plit

In [205]:
csv = CSV.File("/home/stijn/Projects/sudoku_solvers/1000_sudokus.csv"; types=String);
all_puzzles = DataFrame(csv);
random_sudoku_index = 43

random_sudoku = all_puzzles[random_sudoku_index,"quizzes"];
random_sudoku_solution = all_puzzles[random_sudoku_index,"solutions"];

In [206]:
function make_array_from_string(sudoku)
    sudoku_entries_list = map(x -> parse(Int, x),split(sudoku, ""));
    sudoku_row_list = reshape(sudoku_entries_list, (9,9))
end

sudoku = make_array_from_string(random_sudoku)
solution = make_array_from_string(random_sudoku_solution)

9×9 Matrix{Int64}:
 1  9  8  6  5  7  2  3  4
 2  5  4  9  8  3  1  6  7
 3  6  7  1  4  2  9  8  5
 8  4  2  3  9  5  6  7  1
 9  1  6  7  2  4  3  5  8
 7  3  5  8  1  6  4  9  2
 5  8  1  2  3  9  7  4  6
 4  7  9  5  6  1  8  2  3
 6  2  3  4  7  8  5  1  9

In [208]:
function backtrack_sudoku(sudoku, n)
    if n == 81
        # display(sudoku)
        return
    else 
        row = n÷9;
        column = n%9;
        
        block_first_row = 3*(row÷3)+1
        block_first_column = 3*(column÷3)+1
        
        row = row+1
        column = column+1
        
        if sudoku[row, column] == 0
            unavailable_row_values = unique!(sudoku[row,:]);
            unavailable_column_values = unique!(sudoku[:,column]);
            unavailable_block_values = unique!(vec(sudoku[block_first_row:block_first_row+2, block_first_column:block_first_column+2]))
            unavailable_values = union(Set(unavailable_row_values), union(Set(unavailable_column_values), Set(unavailable_block_values)))
            
            available_values = setdiff(Set(1:9),unavailable_values)
            for candidate in available_values
                sudoku[row, column] = candidate
                backtrack_sudoku(sudoku, n+1)
                sudoku[row, column] = 0
            end
        else
            backtrack_sudoku(sudoku, n+1)
        end
    end
end

function solve_sudoku(sudoku)
    backtrack_sudoku(sudoku, 0);
end

solve_sudoku (generic function with 1 method)

In [211]:
@benchmark solve_sudoku(sudoku)
# display(solution)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  363.834 μs …   5.630 ms  ┊ GC (min … max):  0.00% … 92.26%
 Time  (median):     373.419 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   436.578 μs ± 525.971 μs  ┊ GC (mean ± σ):  12.64% ±  9.65%

  █▂                                                            ▁
  ██▇▄▇▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▆ █
  364 μs        Histogram: log(frequency) by time       5.21 ms <

 Memory estimate: 504.19 KiB, allocs estimate: 5130.